In [4]:
flist = (os.listdir('../../cranberry2/Preprocessing/cache/lowess_filtered/preprocess5'))

In [8]:
flist[0]#.split(',')[1:5]

'pacu_5.0,9769,PACU1_3_200606_032535.vital_41'

In [ ]:
[f.split('_')[1] for f in flist]

In [9]:
import statsmodels.api as sm


def smooth(y):
    #return savitzky_golay(y, window_size=2001, order=3)
    return lowess(y)

# 0.2가 제일 잘 없앴음
def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T

import numpy as np
import pandas as pd
import os, pickle, sys
from scipy import signal
#sys.path.append('DL_model')


### input 설정
SRATE = 250 # 250Hz
LEN_INPUT = 20 # input 10s
LEN_PER_NRS = 300 # vital length for each NRS
OVERLAP = 5
n_aug = int((LEN_PER_NRS-LEN_INPUT)/OVERLAP) + 1 # data augmentation 개수


vital_list = pickle.load(open('../Preprocessing/201227_modified_vital','rb'))
input_path = '../DL_model/dataset/preprocess5/'
if not os.path.exists(input_path[:-1]):
    os.mkdir(input_path[:-1])

if os.path.exists(input_path+'x_train_pacu.npz'):
    print('loading train...', flush=True, end='')
    x_train_pacu = np.load(input_path+'x_train_pacu.npz', allow_pickle=True)['arr_0']
    y_train_pacu = np.load(input_path+'y_train_pacu.npz')['arr_0']
    x_test_pacu = np.load(input_path+'x_test_pacu.npz', allow_pickle=True)['arr_0']
    y_test_pacu = np.load(input_path+'y_test_pacu.npz')['arr_0']
    print('done', flush=True)

else:
    false_row_list_preop = []
    
    vital_path = '../../cranberry2/Preprocessing/vital_data/PPG_100Hz_ECG_100Hz_pacu_5min/'
    ecg_path = '../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'
    df_preprocess_pacu = pickle.load(open('../DL_model/cache/preprocess5/df_preprocess_pacu_agender','rb'))
    df_demograph = pickle.load(open('../DL_model/df_caseids+age+gender', 'rb'))
    

    ### test set에 해당하는 file_path
    caseid_test = pickle.load(open('../DL_model/caseid_test_new', 'rb'))
    caseid_train = pickle.load(open('../DL_model/caseid_train_new', 'rb'))
    caseid_val = pickle.load(open('../DL_model/caseid_val_new', 'rb'))
    
    
    non_lis = []
    x_train_pacu, y_train_pacu = [], []
    x_test_pacu, y_test_pacu = [], []
    x_val_pacu, y_val_pacu = [], []
    age_train_pacu, gender_train_pacu = [], []
    age_test_pacu, gender_test_pacu = [], []
    age_val_pacu, gender_val_pacu = [], []

    
    cnt = 0
    for _, row in df_preprocess_pacu.iterrows():
        cnt += 1
        if cnt<=0:
            continue
        if cnt>1000:
            break
            
            
        print('loading data {}/{} ...'.format(cnt, len(df_preprocess_pacu)), end='')
        row_demo = df_demograph[df_demograph['caseids']==row['caseids']].iloc[0]
            

        LEN = LEN_PER_NRS*SRATE
        # vital data - PPG (resampling 100 Hz to 250 Hz)
        df_vital = pickle.load(open(vital_path+row['file_path'], 'rb')).reset_index()
        pleth_samp = df_vital[['PPG']].fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        pleth_resamp = signal.resample(pleth_samp, LEN)
        #ppg_per_NRS = np.full(LEN, np.nan)
        #ppg_per_NRS[-len(pleth_resamp):] = pleth_resamp
        ppg_per_NRS = pleth_resamp
        

        # vital data - ECG (250Hz)
        ecg_samp = pickle.load(open(ecg_path+row['file_path'], 'rb')).reset_index()[['ECG']]
        ecg_samp = ecg_samp.fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        #ecg_per_NRS = np.full(LEN,np.nan)
        #ecg_per_NRS[-len(ecg_samp):] = ecg_samp
        ecg_per_NRS = ecg_samp
        
   
        if len(pleth_resamp) != LEN or len(ecg_samp) != LEN:
            print(row['caseids'])
            break
            

        save_path = '../../cranberry2/Preprocessing/cache/lowess_filtered/preprocess5/pacu_'+row['file_path']
        # 한 NRS에 대해 23개의 input 확인
        for i in range(23,57):
            # input이 전처리 통과한 경우
            if row[str(i+1)][0]:
                if os.path.exists(save_path+'_{}'.format(i)) and row['file_path'] not in vital_list:
                    ppg_inp, ecg_inp = pickle.load(open(save_path+'_{}'.format(i), 'rb'))
                
                else:
                    start_idx = i*OVERLAP*SRATE # 500i
                    end_idx = (i*OVERLAP+LEN_INPUT)*SRATE # 500i + 1000

                    # PPG smoothing
                    ppg_inp = np.copy(ppg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ppg_inp))!=0:
                        ppg_inp = pd.DataFrame(ppg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                    ppg_inp = ppg_inp - smooth(ppg_inp)
                    
                    # ECG smoothing
                    ecg_inp = np.copy(ecg_per_NRS[start_idx:end_idx])
                    if np.sum(np.isnan(ecg_inp))!=0:
                        ecg_inp = pd.DataFrame(ecg_inp).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                    ecg_inp = ecg_inp - smooth(ecg_inp)                    
                
                    # pickle에 저장
                    pickle.dump([ppg_inp, ecg_inp],  open(save_path+'_{}'.format(i), 'wb'))
                    print('{}...'.format(i+1), end='')
        print('done')
'''                
                # normalization
                pleth_inp = ppg_inp - np.nanmean(ppg_inp)
                ecg_inp2 = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)
                
                
                # 해당 caseid가 test set에 속하는 경우
                if row['caseids'] in caseid_test:
                    age_test_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_test_pacu.append(1)
                    else:
                        gender_test_pacu.append(0)
                    x_test_pacu.append([pleth_inp, ecg_inp2])
                    y_test_pacu.append(int(float(row['NRS'])))

                # 해당 caseid가 train set에 해당하는 경우
                elif row['caseids'] in caseid_val:
                    age_val_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_val_pacu.append(1)
                    else:
                        gender_val_pacu.append(0)                    
                    x_val_pacu.append([pleth_inp, ecg_inp2])
                    y_val_pacu.append(int(float(row['NRS'])))
                    
                elif row['caseids'] in caseid_train:
                    age_train_pacu.append(int(row_demo['age']))
                    if row_demo['gender']=='F':
                        gender_train_pacu.append(1)
                    else:
                        gender_train_pacu.append(0)                    
                    x_train_pacu.append([pleth_inp, ecg_inp2])
                    y_train_pacu.append(int(float(row['NRS'])))
                    
                else:
                    non_lis.append(row['caseids'])
                    
        print('completed')
        
        
    x_train_pacu = np.array(x_train_pacu, np.float32)
    x_test_pacu = np.array(x_test_pacu, np.float32)
    y_train_pacu = np.array(y_train_pacu, int)
    y_test_pacu = np.array(y_test_pacu, int)
    x_val_pacu = np.array(x_val_pacu, np.float32)
    y_val_pacu = np.array(y_val_pacu, int)
    
    age_train_pacu = np.array(age_train_pacu, int)
    age_test_pacu = np.array(age_test_pacu, int)
    age_val_pacu = np.array(age_val_pacu, int)
    gender_train_pacu = np.array(gender_train_pacu, int)
    gender_test_pacu = np.array(gender_test_pacu, int)
    gender_val_pacu = np.array(gender_val_pacu, int)
    
        
    # 저장하기
    print('saving...', end='', flush=True)
    np.savez_compressed(input_path+'x_train_pacu2.npz', x_train_pacu)
    np.savez_compressed(input_path+'x_test_pacu2.npz', x_test_pacu)
    np.savez_compressed(input_path+'x_val_pacu2.npz', x_val_pacu)
    np.savez_compressed(input_path+'y_train_pacu2.npz', y_train_pacu)
    np.savez_compressed(input_path+'y_test_pacu2.npz', y_test_pacu)
    np.savez_compressed(input_path+'y_val_pacu2.npz', y_val_pacu)
    
    np.savez_compressed(input_path+'age_train_pacu2.npz', age_train_pacu)
    np.savez_compressed(input_path+'age_test_pacu2.npz', age_test_pacu)
    np.savez_compressed(input_path+'age_val_pacu2.npz', age_val_pacu)    
    np.savez_compressed(input_path+'gender_train_pacu2.npz', gender_train_pacu)
    np.savez_compressed(input_path+'gender_test_pacu2.npz', gender_test_pacu)
    np.savez_compressed(input_path+'gender_val_pacu2.npz', gender_val_pacu)    
    
    print('done', flush=True)

    
    
    
print('size of training set(pacu):', len(x_train_pacu))
print('size of validation set(pacu):', len(x_val_pacu))
print('size of test set(pacu):', len(x_test_pacu))
'''

loading data 1/9949 ...done
loading data 2/9949 ...done
loading data 3/9949 ...done
loading data 4/9949 ...done
loading data 5/9949 ...done
loading data 6/9949 ...done
loading data 7/9949 ...done
loading data 8/9949 ...done
loading data 9/9949 ...done
loading data 10/9949 ...done
loading data 11/9949 ...done
loading data 12/9949 ...done
loading data 13/9949 ...done
loading data 14/9949 ...done
loading data 15/9949 ...done
loading data 16/9949 ...done
loading data 17/9949 ...done
loading data 18/9949 ...done
loading data 19/9949 ...done
loading data 20/9949 ...done
loading data 21/9949 ...done
loading data 22/9949 ...done
loading data 23/9949 ...done
loading data 24/9949 ...done
loading data 25/9949 ...done
loading data 26/9949 ...done
loading data 27/9949 ...done
loading data 28/9949 ...done
loading data 29/9949 ...done
loading data 30/9949 ...done
loading data 31/9949 ...done
loading data 32/9949 ...done
loading data 33/9949 ...done
loading data 34/9949 ...done
loading data 35/9949 ..

loading data 272/9949 ...done
loading data 273/9949 ...done
loading data 274/9949 ...done
loading data 275/9949 ...done
loading data 276/9949 ...done
loading data 277/9949 ...done
loading data 278/9949 ...done
loading data 279/9949 ...done
loading data 280/9949 ...done
loading data 281/9949 ...done
loading data 282/9949 ...done
loading data 283/9949 ...done
loading data 284/9949 ...done
loading data 285/9949 ...done
loading data 286/9949 ...done
loading data 287/9949 ...done
loading data 288/9949 ...done
loading data 289/9949 ...done
loading data 290/9949 ...done
loading data 291/9949 ...done
loading data 292/9949 ...done
loading data 293/9949 ...done
loading data 294/9949 ...done
loading data 295/9949 ...done
loading data 296/9949 ...done
loading data 297/9949 ...done
loading data 298/9949 ...done
loading data 299/9949 ...done
loading data 300/9949 ...done
loading data 301/9949 ...done
loading data 302/9949 ...done
loading data 303/9949 ...done
loading data 304/9949 ...done
loading da

loading data 531/9949 ...24...25...26...27...28...29...30...31...32...33...34...35...40...41...42...done
loading data 532/9949 ...done
loading data 533/9949 ...done
loading data 534/9949 ...done
loading data 535/9949 ...done
loading data 536/9949 ...done
loading data 537/9949 ...done
loading data 538/9949 ...done
loading data 539/9949 ...done
loading data 540/9949 ...done
loading data 541/9949 ...done
loading data 542/9949 ...done
loading data 543/9949 ...done
loading data 544/9949 ...done
loading data 545/9949 ...done
loading data 546/9949 ...done
loading data 547/9949 ...done
loading data 548/9949 ...42...43...44...45...46...47...48...49...50...51...done
loading data 549/9949 ...done
loading data 550/9949 ...35...37...38...39...40...41...42...43...47...done
loading data 551/9949 ...done
loading data 552/9949 ...done
loading data 553/9949 ...done
loading data 554/9949 ...done
loading data 555/9949 ...done
loading data 556/9949 ...done
loading data 557/9949 ...done
loading data 558/994

loading data 795/9949 ...done
loading data 796/9949 ...done
loading data 797/9949 ...done
loading data 798/9949 ...done
loading data 799/9949 ...done
loading data 800/9949 ...done
loading data 801/9949 ...done
loading data 802/9949 ...done
loading data 803/9949 ...done
loading data 804/9949 ...done
loading data 805/9949 ...done
loading data 806/9949 ...done
loading data 807/9949 ...done
loading data 808/9949 ...done
loading data 809/9949 ...done
loading data 810/9949 ...done
loading data 811/9949 ...done
loading data 812/9949 ...done
loading data 813/9949 ...done
loading data 814/9949 ...done
loading data 815/9949 ...done
loading data 816/9949 ...done
loading data 817/9949 ...done
loading data 818/9949 ...done
loading data 819/9949 ...done
loading data 820/9949 ...done
loading data 821/9949 ...done
loading data 822/9949 ...done
loading data 823/9949 ...done
loading data 824/9949 ...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...

"                \n                # normalization\n                pleth_inp = ppg_inp - np.nanmean(ppg_inp)\n                ecg_inp2 = (ecg_inp - np.nanmean(ecg_inp)) / np.nanstd(ecg_inp)\n                \n                \n                # 해당 caseid가 test set에 속하는 경우\n                if row['caseids'] in caseid_test:\n                    age_test_pacu.append(int(row_demo['age']))\n                    if row_demo['gender']=='F':\n                        gender_test_pacu.append(1)\n                    else:\n                        gender_test_pacu.append(0)\n                    x_test_pacu.append([pleth_inp, ecg_inp2])\n                    y_test_pacu.append(int(float(row['NRS'])))\n\n                # 해당 caseid가 train set에 해당하는 경우\n                elif row['caseids'] in caseid_val:\n                    age_val_pacu.append(int(row_demo['age']))\n                    if row_demo['gender']=='F':\n                        gender_val_pacu.append(1)\n                    else:\n          